In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
!pip install contractions
import contractions
import re
import string
import operator
import nltk
from nltk.corpus import stopwords

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.1 MB/s eta 0:00:00
/kaggle/input/glove840b300dtxt/glove.840B.300d.txt
/kaggle/input/paragram-300-sl999/paragram_300_sl999.txt
/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/wikinews300d1msubwordvec/wiki-news-300d-1M-subword.vec


In [2]:
train = pd.read_csv('../input/nlp-getting-started/train.csv')
test = pd.read_csv('../input/nlp-getting-started/test.csv')

In [3]:
print('Training data shape: ', train.shape)
print('Test data shape: ', test.shape)

Training data shape:  (7613, 5)
Test data shape:  (3263, 4)


In [4]:
df = pd.concat([train, test])
print('Concat data shape: ', df.shape)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
df[301:311]

Concat data shape:  (10876, 5)


,id,keyword,location,text,target
301,443,apocalypse,NaN,Short Reading\n\nApocalypse 21:1023 \n\nIn the spirit the angel took me to the top of an enormous high mountain and... http://t.co/v8AfTD9zeZ,1.0
302,444,apocalypse,Tokyo,Enjoyed live-action Attack on Titan but every time I see posters I'm reminded how freshly clean and coiffed everyone is in the apocalypse.,0.0
303,445,apocalypse,NaN,I liked a @YouTube video http://t.co/ki1yKrs9fi Minecraft: NIGHT LUCKY BLOCK MOD (BOB APOCALYPSE WITHER 2.0 &amp; MORE!) Mod Showcase,0.0
304,446,armageddon,"California, United States",#PBBan (Temporary:300) avYsss @'aRmageddon | DO NOT KILL | FLAGS ONLY | Fast XP' for Reason,0.0
305,447,armageddon,"California, United States",#PBBan (Temporary:300) Russaky89 @'aRmageddon | DO NOT KILL | FLAGS ONLY | Fast XP' for Reason,0.0
306,448,armageddon,#FLIGHTCITY UK,((OFFICIAL VID)) #DoubleCups &gt;&gt; https://t.co/lfKMTZaEkk &gt;&gt; @TrubGME Prod @THISIZBWRIGHT &gt;&gt; #ARMAGEDDON,0.0
307,450,armageddon,NaN,ouvindo Peace Love &amp; Armageddon,0.0
308,451,armageddon,NaN,Best movie you've ever seen? - Armageddon http://t.co/qoUXIgdtbZ,0.0
309,452,armageddon,"Alphen aan den Rijn, Holland",Bed time. Don't wake me up unless revolution or Armageddon start.,0.0
310,453,armageddon,NaN,Red Faction: Armageddon (Microsoft Xbox 360 2011) - Full read by eBay http://t.co/ypbVS1IJya http://t.co/9dFLv6ynqr,0.0


**Embeddings**

In [5]:
# functions for loading embeddings and checking embedding coverage
def load_embed(file):
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    
    if file == '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file) if len(o)>100)
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
        
    return embeddings_index

def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words

In [6]:
glove = '../input/glove840b300dtxt/glove.840B.300d.txt'
paragram =  '../input/paragram-300-sl999/paragram_300_sl999.txt'
wiki_news = '../input/wikinews300d1msubwordvec/wiki-news-300d-1M-subword.vec'
embed_glove = load_embed(glove)
embed_paragram = load_embed(paragram)
embed_fasttext = load_embed(wiki_news)

**Coverage Before Pre-processing**

In [7]:
vocab = build_vocab(df['text'])
print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)
print("Paragram : ")
oov_paragram = check_coverage(vocab, embed_paragram)
print("FastText : ")
oov_fasttext = check_coverage(vocab, embed_fasttext)

Glove : 
Found embeddings for 48.73% of vocab
Found embeddings for  82.43% of all text
Paragram : 
Found embeddings for 20.61% of vocab
Found embeddings for  56.08% of all text
FastText : 
Found embeddings for 45.38% of vocab
Found embeddings for  79.45% of all text


**Pre-processing**

Text Cleaning

In [8]:
def clean_text(text):
    # lowercase
    text = text.lower()
    # expand contractions
    text = contractions.fix(text)
    # remove urls
    text = re.sub('https?://\S+|www\.\S+', ' ', text)
    # remove HTML tags
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    text = re.sub(html, ' ', text)
    # remove irrelevant characters - punctuations and numbers
    #text = re.sub(r'[^a-zA-Z]', ' ', text)
    # remove punctuations
    #table = str.maketrans('','',string.punctuation)
    #text = text.translate(table)
    text = re.sub('['+string.punctuation+']', ' ', text)
    # remove numbers
    text = re.sub(r'\d+', ' ', text)
    # remove non-ASCII characters
    text = re.sub(r'[^\x00-\x7f]',r' ', text)
    # remove newline \n
    text = re.sub('\n', ' ', text)
    # remove special characters
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    # remove extra whitespaces
    pattern = r'^\s*|\s\s*'
    text = re.sub(pattern, ' ', text).strip()
    
    return text

In [9]:
df[301:311]

,id,keyword,location,text,target
301,443,apocalypse,NaN,Short Reading\n\nApocalypse 21:1023 \n\nIn the spirit the angel took me to the top of an enormous high mountain and... http://t.co/v8AfTD9zeZ,1.0
302,444,apocalypse,Tokyo,Enjoyed live-action Attack on Titan but every time I see posters I'm reminded how freshly clean and coiffed everyone is in the apocalypse.,0.0
303,445,apocalypse,NaN,I liked a @YouTube video http://t.co/ki1yKrs9fi Minecraft: NIGHT LUCKY BLOCK MOD (BOB APOCALYPSE WITHER 2.0 &amp; MORE!) Mod Showcase,0.0
304,446,armageddon,"California, United States",#PBBan (Temporary:300) avYsss @'aRmageddon | DO NOT KILL | FLAGS ONLY | Fast XP' for Reason,0.0
305,447,armageddon,"California, United States",#PBBan (Temporary:300) Russaky89 @'aRmageddon | DO NOT KILL | FLAGS ONLY | Fast XP' for Reason,0.0
306,448,armageddon,#FLIGHTCITY UK,((OFFICIAL VID)) #DoubleCups &gt;&gt; https://t.co/lfKMTZaEkk &gt;&gt; @TrubGME Prod @THISIZBWRIGHT &gt;&gt; #ARMAGEDDON,0.0
307,450,armageddon,NaN,ouvindo Peace Love &amp; Armageddon,0.0
308,451,armageddon,NaN,Best movie you've ever seen? - Armageddon http://t.co/qoUXIgdtbZ,0.0
309,452,armageddon,"Alphen aan den Rijn, Holland",Bed time. Don't wake me up unless revolution or Armageddon start.,0.0
310,453,armageddon,NaN,Red Faction: Armageddon (Microsoft Xbox 360 2011) - Full read by eBay http://t.co/ypbVS1IJya http://t.co/9dFLv6ynqr,0.0


In [10]:
df['text'] = df['text'].apply(lambda x: clean_text(x))

In [11]:
df[301:311]

,id,keyword,location,text,target
301,443,apocalypse,NaN,short reading apocalypse in the spirit the angel took me to the top of an enormous high mountain and,1.0
302,444,apocalypse,Tokyo,enjoyed live action attack on titan but every time i see posters i am reminded how freshly clean and coiffed everyone is in the apocalypse,0.0
303,445,apocalypse,NaN,i liked a youtube video minecraft night lucky block mod bob apocalypse wither more mod showcase,0.0
304,446,armageddon,"California, United States",pbban temporary avysss armageddon do not kill flags only fast xp for reason,0.0
305,447,armageddon,"California, United States",pbban temporary russaky armageddon do not kill flags only fast xp for reason,0.0
306,448,armageddon,#FLIGHTCITY UK,official vid doublecups trubgme prod thisizbwright armageddon,0.0
307,450,armageddon,NaN,ouvindo peace love armageddon,0.0
308,451,armageddon,NaN,best movie you have ever seen armageddon,0.0
309,452,armageddon,"Alphen aan den Rijn, Holland",bed time do not wake me up unless revolution or armageddon start,0.0
310,453,armageddon,NaN,red faction armageddon microsoft xbox full read by ebay,0.0


Remove Stopwords

In [12]:
nltk.download('stopwords')
sw = stopwords.words('english')

def remove_stopwords(text):
    text = ' '.join([word for word in text.split() if word not in sw])
    return text

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
df[301:311]

,id,keyword,location,text,target
301,443,apocalypse,NaN,short reading apocalypse in the spirit the angel took me to the top of an enormous high mountain and,1.0
302,444,apocalypse,Tokyo,enjoyed live action attack on titan but every time i see posters i am reminded how freshly clean and coiffed everyone is in the apocalypse,0.0
303,445,apocalypse,NaN,i liked a youtube video minecraft night lucky block mod bob apocalypse wither more mod showcase,0.0
304,446,armageddon,"California, United States",pbban temporary avysss armageddon do not kill flags only fast xp for reason,0.0
305,447,armageddon,"California, United States",pbban temporary russaky armageddon do not kill flags only fast xp for reason,0.0
306,448,armageddon,#FLIGHTCITY UK,official vid doublecups trubgme prod thisizbwright armageddon,0.0
307,450,armageddon,NaN,ouvindo peace love armageddon,0.0
308,451,armageddon,NaN,best movie you have ever seen armageddon,0.0
309,452,armageddon,"Alphen aan den Rijn, Holland",bed time do not wake me up unless revolution or armageddon start,0.0
310,453,armageddon,NaN,red faction armageddon microsoft xbox full read by ebay,0.0


In [14]:
df['text'] = df['text'].apply(lambda x: remove_stopwords(x))

In [15]:
df[301:311]

,id,keyword,location,text,target
301,443,apocalypse,NaN,short reading apocalypse spirit angel took top enormous high mountain,1.0
302,444,apocalypse,Tokyo,enjoyed live action attack titan every time see posters reminded freshly clean coiffed everyone apocalypse,0.0
303,445,apocalypse,NaN,liked youtube video minecraft night lucky block mod bob apocalypse wither mod showcase,0.0
304,446,armageddon,"California, United States",pbban temporary avysss armageddon kill flags fast xp reason,0.0
305,447,armageddon,"California, United States",pbban temporary russaky armageddon kill flags fast xp reason,0.0
306,448,armageddon,#FLIGHTCITY UK,official vid doublecups trubgme prod thisizbwright armageddon,0.0
307,450,armageddon,NaN,ouvindo peace love armageddon,0.0
308,451,armageddon,NaN,best movie ever seen armageddon,0.0
309,452,armageddon,"Alphen aan den Rijn, Holland",bed time wake unless revolution armageddon start,0.0
310,453,armageddon,NaN,red faction armageddon microsoft xbox full read ebay,0.0


**Coverage After Pre-processing**

In [16]:
vocab = build_vocab(df['text'])
print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)
print("Paragram : ")
oov_paragram = check_coverage(vocab, embed_paragram)
print("FastText : ")
oov_fasttext = check_coverage(vocab, embed_fasttext)

Glove : 
Found embeddings for 75.52% of vocab
Found embeddings for  93.76% of all text
Paragram : 
Found embeddings for 64.37% of vocab
Found embeddings for  89.56% of all text
FastText : 
Found embeddings for 69.17% of vocab
Found embeddings for  91.87% of all text
